In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import linalg
from datetime import datetime
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from openpyxl import Workbook
import os

In [4]:
# Load and clean data
hf_df = pd.read_excel("Data/hedge_funds_returns_data.xlsx")
factors_df = pd.read_excel("Data/factors_returns_data.xlsx")

short_mapping = {
    'Date': 'Date',
    'HFRI 400 (US) Fund Weighted Composite Index (HFRI4FWC)': 'HFRI4FWC',
    'HFRI 400 (US) EH: Long/Short Index (HFRI4ELS)': 'HFRI4ELS',
    'HFRI 400 (US) EH: Fundamental Value Index (HFRI4EHV)': 'HFRI4EHV',
    'HFRI 400 (US) Event-Driven Index (HFRI4ED)': 'HFRI4ED'
    
}

hf_df = hf_df.rename(columns=short_mapping)

def clean_data(df):
    df = df.set_index('Date')
    df = df.dropna()
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

hf_df_clean = clean_data(hf_df)
factors_df_clean = clean_data(factors_df)

print(f"Data loaded: HF shape {hf_df_clean.shape}, Factors shape {factors_df_clean.shape}")

Data loaded: HF shape (221, 4), Factors shape (221, 5)


In [5]:
# Factor model functions
def estimate_factor_betas(hf_data, factor_data):
    factor_cols = ['Mkt-RF', 'SMB', 'HML', 'RF']
    if 'Mom' in factor_data.columns:
        factor_cols.insert(3, 'Mom')
    elif 'Mom   ' in factor_data.columns:
        factor_cols.insert(3, 'Mom   ')
    
    factor_loadings = pd.DataFrame(index=hf_data.columns, columns=factor_cols)
    
    for hf_name in hf_data.columns:
        y = hf_data[hf_name].dropna()
        X = factor_data.loc[y.index, factor_cols].dropna()
        common_idx = y.index.intersection(X.index)
        y_aligned = y.loc[common_idx]
        X_aligned = X.loc[common_idx]
        
        if len(y_aligned) > 24:
            reg = LinearRegression()
            reg.fit(X_aligned, y_aligned)
            factor_loadings.loc[hf_name] = reg.coef_
    
    return factor_loadings

def estimate_alphas(hf_data, factor_data):
    factor_cols = ['Mkt-RF', 'SMB', 'HML', 'RF']
    if 'Mom' in factor_data.columns:
        factor_cols.insert(3, 'Mom')
    elif 'Mom   ' in factor_data.columns:
        factor_cols.insert(3, 'Mom   ')
    
    alphas = pd.Series(index=hf_data.columns, name='Alpha')
    
    for hf_name in hf_data.columns:
        y = hf_data[hf_name].dropna()
        X = factor_data.loc[y.index, factor_cols].dropna()
        common_idx = y.index.intersection(X.index)
        y_aligned = y.loc[common_idx]
        X_aligned = X.loc[common_idx]
        
        if len(y_aligned) > 24:
            reg = LinearRegression()
            reg.fit(X_aligned, y_aligned)
            alphas.loc[hf_name] = reg.intercept_
    
    return alphas

def calculate_residuals(hf_data, factor_data, factor_loadings, alphas):
    mom_col = 'Mom' if 'Mom' in factor_data.columns else 'Mom   '
    residuals = pd.DataFrame(index=hf_data.index, columns=hf_data.columns)
    
    for hf_name in hf_data.columns:
        if hf_name in factor_loadings.index and pd.notna(alphas.loc[hf_name]):
            actual_returns = hf_data[hf_name]
            predicted_returns = (
                alphas.loc[hf_name] +
                factor_loadings.loc[hf_name, 'Mkt-RF'] * factor_data['Mkt-RF'] +
                factor_loadings.loc[hf_name, 'SMB'] * factor_data['SMB'] +
                factor_loadings.loc[hf_name, 'HML'] * factor_data['HML'] +
                factor_loadings.loc[hf_name, mom_col] * factor_data[mom_col] +
                factor_loadings.loc[hf_name, 'RF'] * factor_data['RF']
            )
            residuals[hf_name] = actual_returns - predicted_returns
    
    return residuals

In [6]:
# Noise filtering functions
def calculate_rmt_thresholds(residuals_df):
    T, N = residuals_df.dropna().shape
    c = T / N
    lambda_max = (1 + np.sqrt(c))**2
    lambda_min = (1 - np.sqrt(c))**2
    return {'T': T, 'N': N, 'c_ratio': c, 'lambda_max_theory': lambda_max, 'lambda_min_theory': lambda_min}

def apply_tyler_m_estimator(residuals_df, max_iter=100, tol=1e-6):
    X = residuals_df.dropna().values
    T, N = X.shape
    X_centered = X - np.mean(X, axis=0)
    cov_est = np.cov(X_centered.T)
    
    for iteration in range(max_iter):
        cov_old = cov_est.copy()
        try:
            cov_inv = np.linalg.inv(cov_est)
            weights = np.zeros(T)
            
            for t in range(T):
                x_t = X_centered[t, :]
                weights[t] = 1.0 / np.sqrt(x_t @ cov_inv @ x_t + 1e-12)
                
            weighted_sum = np.zeros((N, N))
            weight_sum = 0
            
            for t in range(T):
                x_t = X_centered[t, :].reshape(-1, 1)
                weighted_sum += weights[t] * (x_t @ x_t.T)
                weight_sum += weights[t]
                
            cov_est = N * weighted_sum / weight_sum
            
            diff = np.linalg.norm(cov_est - cov_old, 'fro')
            if diff < tol:
                converged = True
                break
                
        except np.linalg.LinAlgError:
            cov_est += 1e-6 * np.eye(N)
            continue
    else:
        converged = False
        
    convergence_info = {'converged': converged, 'iterations': iteration + 1}
    robust_cov = pd.DataFrame(cov_est, index=residuals_df.columns, columns=residuals_df.columns)
    return robust_cov, convergence_info

def separate_signal_noise_eigenvalues(covariance_matrix, rmt_thresholds):
    cov_matrix = covariance_matrix.values if hasattr(covariance_matrix, 'values') else covariance_matrix
    eigenvalues, eigenvectors = linalg.eigh(cov_matrix)
    eigenvalues = eigenvalues[::-1]
    eigenvectors = eigenvectors[:, ::-1]
    
    N = len(eigenvalues)
    avg_variance = np.trace(cov_matrix) / N
    scaled_eigenvals = eigenvalues / avg_variance
    
    lambda_max = min(rmt_thresholds['lambda_max_theory'], 3.0)
    signal_mask = scaled_eigenvals > lambda_max
    
    if np.sum(signal_mask) == 0:
        signal_mask[:2] = True
    
    n_signal = np.sum(signal_mask)
    n_noise = N - n_signal
    
    return {
        'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors, 'signal_mask': signal_mask,
        'n_signal': n_signal, 'n_noise': n_noise, 'rmt_threshold': lambda_max
    }

def reconstruct_filtered_covariance(separation_results):
    eigenvalues = separation_results['eigenvalues']
    eigenvectors = separation_results['eigenvectors']
    signal_mask = separation_results['signal_mask']
    
    filtered_eigenvals = eigenvalues.copy()
    if separation_results['n_noise'] > 0:
        noise_eigenvals = eigenvalues[~signal_mask]
        avg_noise = np.mean(noise_eigenvals)
        filtered_eigenvals[~signal_mask] = avg_noise
    
    filtered_cov = eigenvectors @ np.diag(filtered_eigenvals) @ eigenvectors.T
    eigenvals, eigenvecs = linalg.eigh(filtered_cov)
    eigenvals = np.maximum(eigenvals, 1e-8)
    filtered_cov = eigenvecs @ np.diag(eigenvals) @ eigenvecs.T
    
    return filtered_cov, {'method': 'average_noise'}

In [7]:
# Rolling window functions
def get_quarter_info(date):
    year = date.year
    month = date.month
    if month <= 3:
        quarter = f"Q1_{year}"
        quarter_start = pd.Timestamp(year, 1, 1)
    elif month <= 6:
        quarter = f"Q2_{year}"
        quarter_start = pd.Timestamp(year, 4, 1)
    elif month <= 9:
        quarter = f"Q3_{year}"
        quarter_start = pd.Timestamp(year, 7, 1)
    else:
        quarter = f"Q4_{year}"
        quarter_start = pd.Timestamp(year, 10, 1)
    return quarter, quarter_start

def generate_rolling_periods(hf_data, factors_data, training_years=7):
    start_date = max(hf_data.index.min(), factors_data.index.min())
    end_date = min(hf_data.index.max(), factors_data.index.max())
    periods = []
    
    first_quarter_start = start_date + pd.DateOffset(years=training_years)
    current_date = pd.Timestamp(first_quarter_start.year, ((first_quarter_start.month-1)//3)*3 + 1, 1)
    
    while current_date <= end_date:
        quarter, quarter_start = get_quarter_info(current_date)
        train_start = quarter_start - pd.DateOffset(years=training_years)
        train_end = quarter_start - pd.DateOffset(days=1)
        
        hf_train_data = hf_data[(hf_data.index >= train_start) & (hf_data.index <= train_end)]
        factors_train_data = factors_data[(factors_data.index >= train_start) & (factors_data.index <= train_end)]
        
        if len(hf_train_data) >= 60 and len(factors_train_data) >= 60:
            periods.append({
                'quarter': quarter,
                'hf_data': hf_train_data,
                'factors_data': factors_train_data
            })
        
        if current_date.month <= 3:
            current_date = pd.Timestamp(current_date.year, 4, 1)
        elif current_date.month <= 6:
            current_date = pd.Timestamp(current_date.year, 7, 1)
        elif current_date.month <= 9:
            current_date = pd.Timestamp(current_date.year, 10, 1)
        else:
            current_date = pd.Timestamp(current_date.year + 1, 1, 1)
    
    return periods

In [8]:
# Main calculation function
def calculate_single_filtered_covariance(hf_data, factors_data):
    try:
        factor_betas = estimate_factor_betas(hf_data, factors_data)
        alphas = estimate_alphas(hf_data, factors_data)
        residuals = calculate_residuals(hf_data, factors_data, factor_betas, alphas)
        
        residuals_clean = residuals.dropna()
        if residuals_clean.empty or residuals_clean.shape[0] < 24:
            return None
        
        rmt_thresholds = calculate_rmt_thresholds(residuals_clean)
        robust_cov, convergence_info = apply_tyler_m_estimator(residuals_clean)
        separation_results = separate_signal_noise_eigenvalues(robust_cov, rmt_thresholds)
        filtered_cov, reconstruction_info = reconstruct_filtered_covariance(separation_results)
        
        return filtered_cov
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def calculate_rolling_filtered_covariance_matrices(hf_data, factors_data):
    periods = generate_rolling_periods(hf_data, factors_data)
    covariance_matrices = {}
    
    print(f"Processing {len(periods)} quarters...")
    
    for period in tqdm(periods):
        quarter = period['quarter']
        filtered_cov = calculate_single_filtered_covariance(period['hf_data'], period['factors_data'])
        
        if filtered_cov is not None:
            var_name = f"filtered_cov_{quarter}"
            covariance_matrices[var_name] = filtered_cov
    
    print(f"Successfully calculated {len(covariance_matrices)} covariance matrices")
    return covariance_matrices

In [9]:
# Execute calculation and create easy access variables
covariance_matrices = calculate_rolling_filtered_covariance_matrices(hf_df_clean, factors_df_clean)

# Store in global namespace for easy access
globals().update(covariance_matrices)

# Show available quarters
available_quarters = sorted([key.replace('filtered_cov_', '') for key in covariance_matrices.keys()])
print(f"\n✓ Created {len(covariance_matrices)} covariance matrix variables")
print(f"Available quarters: {available_quarters[:10]}{'...' if len(available_quarters) > 10 else ''}")

Processing 46 quarters...


100%|██████████| 46/46 [00:00<00:00, 60.14it/s]

Successfully calculated 46 covariance matrices

✓ Created 46 covariance matrix variables
Available quarters: ['Q1_2012', 'Q1_2013', 'Q1_2014', 'Q1_2015', 'Q1_2016', 'Q1_2017', 'Q1_2018', 'Q1_2019', 'Q1_2020', 'Q1_2021']...


In [10]:
covariance_matrices

{'filtered_cov_Q1_2012': array([[0.00044537, 0.0004898 , 0.00033357, 0.00037073],
        [0.0004898 , 0.00073731, 0.00044748, 0.00049733],
        [0.00033357, 0.00044748, 0.000385  , 0.0003387 ],
        [0.00037073, 0.00049733, 0.0003387 , 0.00045668]]),
 'filtered_cov_Q2_2012': array([[0.00042692, 0.00046378, 0.00032035, 0.00034928],
        [0.00046378, 0.00069565, 0.00042634, 0.00046483],
        [0.00032035, 0.00042634, 0.00037292, 0.00032108],
        [0.00034928, 0.00046483, 0.00032108, 0.0004285 ]]),
 'filtered_cov_Q3_2012': array([[0.00041169, 0.00044243, 0.00030512, 0.00033097],
        [0.00044243, 0.00066563, 0.000405  , 0.00043931],
        [0.00030512, 0.000405  , 0.00035767, 0.00030296],
        [0.00033097, 0.00043931, 0.00030296, 0.000407  ]]),
 'filtered_cov_Q4_2012': array([[0.0003888 , 0.00041343, 0.00029444, 0.00030765],
        [0.00041343, 0.00061795, 0.00038714, 0.0004045 ],
        [0.00029444, 0.00038714, 0.00035008, 0.00028808],
        [0.00030765, 0.00040

In [11]:
output_folder = 'all_output_results'
os.makedirs(output_folder, exist_ok=True)
excel_filename = os.path.join(output_folder, "Noise_filtered_covariance.xlsx")

with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    
    # Get hedge fund column names for proper indexing
    hf_columns = hf_df_clean.columns.tolist()
    
    # Iterate through each covariance matrix
    for var_name, cov_matrix in covariance_matrices.items():
        # Extract quarter name from variable name (remove 'filtered_cov_' prefix)
        sheet_name = var_name.replace('filtered_cov_', '')
        
        # Convert numpy array to pandas DataFrame with proper index and column names
        cov_df = pd.DataFrame(
            cov_matrix, 
            index=hf_columns, 
            columns=hf_columns
        )
        
        # Write to Excel sheet
        cov_df.to_excel(writer, sheet_name=sheet_name, index=True)
        
        print(f"✓ Exported {sheet_name} sheet")

print(f"\n✅ Successfully exported {len(covariance_matrices)} covariance matrices to '{excel_filename}'")
print(f"📁 File saved in: {excel_filename}")

# Display summary of exported sheets
sheet_names = [key.replace('filtered_cov_', '') for key in covariance_matrices.keys()]
print(f"\n📊 Exported sheets: {sorted(sheet_names)}")

✓ Exported Q1_2012 sheet
✓ Exported Q2_2012 sheet
✓ Exported Q3_2012 sheet
✓ Exported Q4_2012 sheet
✓ Exported Q1_2013 sheet
✓ Exported Q2_2013 sheet
✓ Exported Q3_2013 sheet
✓ Exported Q4_2013 sheet
✓ Exported Q1_2014 sheet
✓ Exported Q2_2014 sheet
✓ Exported Q3_2014 sheet
✓ Exported Q4_2014 sheet
✓ Exported Q1_2015 sheet
✓ Exported Q2_2015 sheet
✓ Exported Q3_2015 sheet
✓ Exported Q4_2015 sheet
✓ Exported Q1_2016 sheet
✓ Exported Q2_2016 sheet
✓ Exported Q3_2016 sheet
✓ Exported Q4_2016 sheet
✓ Exported Q1_2017 sheet
✓ Exported Q2_2017 sheet
✓ Exported Q3_2017 sheet
✓ Exported Q4_2017 sheet
✓ Exported Q1_2018 sheet
✓ Exported Q2_2018 sheet
✓ Exported Q3_2018 sheet
✓ Exported Q4_2018 sheet
✓ Exported Q1_2019 sheet
✓ Exported Q2_2019 sheet
✓ Exported Q3_2019 sheet
✓ Exported Q4_2019 sheet
✓ Exported Q1_2020 sheet
✓ Exported Q2_2020 sheet
✓ Exported Q3_2020 sheet
✓ Exported Q4_2020 sheet
✓ Exported Q1_2021 sheet
✓ Exported Q2_2021 sheet
✓ Exported Q3_2021 sheet
✓ Exported Q4_2021 sheet
